# Data Linkage

**Tian Lou** \
Ohio Education Research Center \
The Ohio State University

**Xiangyu Ren** \
New York University

**Anna-Carolina Haensch** \
University of Maryland \
LMU Munich

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.10263021.svg)](https://doi.org/10.5281/zenodo.10263021)

**This notebook is developed for the [Data Literacy and Evidence Building Executive Class](https://www.socialdatascience.umd.edu/data-literacy).**

**The "Syntucky" data, which is synthetic in nature, is exclusively designed for training exercises. It is not intended to derive meaningful insights or make determinations about real-world populations.**

## Goals

Data linkage is a key part of data science. It involves combining multiple data sources to get a more comprehensive understanding of an individual's experience. Recall that in the Data Exploration notebook, we explored the 2015 Syntucky cohort data, which includes demographic information, college enrollment and graduation, and labor market outcomes of degree pursuers who entered college for the first time in 2015. While in this class we have convenient access to all these pieces of information in one dataset, in reality, we need to do a tremedous amount of work to pre-process, clean, format, and link various education and employment datasets. 

In this notebook, we introduce you to a few data linkage methods and show you how different methods can generate different samples. Specifically, we will link three datasets about *Syntucky students who enrolled in college in 2015*: 1) a **master dataset**, which contains demographic information; 2) an **employment dataset**, which reveals labor market information during the seventh year after college entrance; 3) an **education dataset**, which provides details about a student's higher education background.

**The specific questions we seek to answer in this notebook are**:
1. What are the differences between left, right, and inner joins? 
2. How do different join methods affect the sample size and missingness of the final data?
3. When do we need to use left/right/inner join?

**After completing this notebook, you should:**
1. Learn how to join cross-sectional datasets in order to create a dataset that contains the key information about an individual.
2. Become familar with SQL commands, `LEFT JOIN` and `INNER JOIN`, and Python Pandas function, `merge()`.
3. Understand how different join methods would affect your final data.

## 1. Import Data

In [ ]:
# Load libraries

# Interface to connect mySQL database server in Python
import MySQLdb 

# Library that provides lightweight disk-based database
import sqlite3 

# File system path
from pathlib import Path

# Data manipulation and analysis tool
import pandas as pd 

As mentioned above, in order to create a dataset that includes a person's demographics, education, and employment histories, we will link three datasets:  

**Master Dataset**: This dataset contains **demographic information about the synthetic individuals that enrolled in college in 2015**. (They may enter college during any calendar year between 2005 and 2015). The variables in this dataset are time-invariant, meaning they do not change over time. It includes birth year, birth month, gender, race, underrepresented minority (URM) status, and in-state origin indicator. 

**Employment Dataset**: This dataset provides information about **Syntucky individuals' labor market status** during the seventh year after college entrance. **This dataset only includes people who were employed during the seventh year after college entrance**. 

**Education Dataset**: This dataset holds information regarding **college entrance, enrollment, and degrees obtained**. It includes details such as the year a person first entered college as degree pursuers, the majors they enrolled in, the highest degree they obtained, etc.

Through the process of joining these datasets, a more comprehensive and cohesive depiction of the synthetic units' life journey can be obtained. The simple **ER diagram** below shows the relationships between the three datasets. We can see that the common column that we can use to join any two of these datasets should be `id`.

![diagram.png](diagram.png)

Before running the code below, please change <font color='red'> **YOUR DATA DIRECTORY**</font> to your own file path.

In [ ]:
#Define data directory
data_directory = 'YOUR DATA DIRECTORY'

#master data
master_df = pd.read_csv(data_directory + 'master_crosssection.csv')

#employment data
employment_df = pd.read_csv(data_directory + 'employment_crosssection.csv')

#education data
education_df = pd.read_csv(data_directory + 'education_crosssection.csv')

Now, let's take a close look at the number of rows in each dataset and what columns they contain.

In [ ]:
#Master data
print('The master data has', master_df.shape[0], 'rows.')

master_df.head()

In [ ]:
#Employment data
print('The employment data has', employment_df.shape[0], 'rows.')

employment_df.head()

In [ ]:
#Education data
print('The education data has', education_df.shape[0], 'rows.')

education_df.head()

## 2. Establish Database Connection and Load Data to the Database

Utilizing SQL in Python opens up new dimensions for data handling and manipulation, providing powerful querying capabilities. Here's the process we use to transfer data from a CSV file to a database:

1. **Create a Database**: We use Python code `Path ('C:/Users/*YOUR FOLDERNAME*/syn_data.db').touch()` to create a new SQLite database file named `syn_data.db`. It won't modify the file if it does exist, making this a safe operation to perform even if you're unsure about the file's existence. 

> Before running the code below, please change <font color='red'> **YOUR USERNAME**</font> to your username or use your own file path.

In [ ]:
# Change your working path to your personal folder
Path('C:/Users/YOUR USERNAME/syn_data.db').touch() 

2. **Establish the Database Connection**: Then we need to tell Python to connect to our database, which is like opening a book to start reading it. Once our "book" (database) is open, we need a way to navigate through it. In Python, we do this by using `cursor()`. You can think of the cursor like a finger pointing at a line in the book, ready to read or change the words. The code `c = conn.cursor()` creates the cursor and the abbreviation of the cursor is `c`.

 > Before running the code below, please change <font color='red'> **YOUR USERNAME**</font> to your username or use your own file path.

In [ ]:
# Establish database connection
conn = sqlite3.connect('C:/Users/YOUR USERNAME/syn_data.db') 
c = conn.cursor()

3. **Create a Table**: The next step is to create a table within the database to house the data from our CSV files. We use `CREATE TABLE tblname` to create tables, where `tblname` is the name of your table. This process requires a list of column definitions, including column name and type.

   The code below, `c.execute('''CREATE TABLE master(...) ''')`, runs a SQL query that creates a new table named *master* in our database. In the *master* table, we are specifying six columns: *id*, *gender*, *birth_year*, *birth_month*, *urm_status*, *race_group*, and *instate_origin*. We assign each of these columns a specific data type: *id*, *birth_year*, and *instate_origin* are integers, represented by `int`, while *gender*, *birth_month*, *urm_status*, and *race_group* will contain text, represented by `text`. The command is wrapped in triple quotes, which is a Python convention for defining multi-line strings, but in this case, it's used for a single line of string.

In [ ]:
#Remove the table if already exist
c.execute('''DROP TABLE IF EXISTS master ''')

#Create an empty table, "master", in your database
#In the code below, we define the column names and types before we can upload the master data to the database
c.execute('''CREATE TABLE master (id int, 
                                  gender text, 
                                  birth_year int, 
                                  birth_month text,
                                  urm_status text,
                                  race_group text,
                                  instate_origin  int)''')

4. **Load Data**: The next step is to populate the table with data. The code `master_df.to_sql('master',conn, if_exists = 'replace',index = False)` is a method from Pandas. It is used to write records stored in a DataFrame to a SQL table. It takes several arguments, including the name of the SQL table, the connection instance to the database, and some optional parameters. In this case, `if_exists = 'replace'` indicates that if the table already exists in the database, the new data should replace the existing data. The `index = False` argument tells the function not to write row indices from the DataFrame into the SQL table. So, this command is essentially saying, "Take the data in our `master_df` DataFrame and replace the old table with the *master* table in our database, but don't worry about including the DataFrame's indices."

In [ ]:
#Load data in master_df to the database
master_df.to_sql('master', conn, if_exists = 'replace', index = False)

By following  the above steps, we'll be able to effectively move our data into a SQL database and execute SQL commands within our Python environment. Now let's repeat the above steps for the employment and education datasets.

In [ ]:
#Remove the table if already exist
c.execute('''DROP TABLE IF EXISTS employment ''')

#Create the employment table in the database
c.execute('''CREATE TABLE employment (id int,
                                      year7_max_qtrs_one_employer int, 
                                      year7_education_industry_employed int, 
                                      year7_ct_qtrs_employed int,
                                      year7_ct_employers int, 
                                      year7_earnings int, 
                                      year7_earnings_most_consistent_employer int)''')

In [ ]:
#Load employment data to the database
employment_df.to_sql('employment', conn, if_exists = 'replace', index = False)

In [ ]:
#Remove the table if already exist
c.execute('''DROP TABLE IF EXISTS education ''')

#Create the education table in the database
c.execute('''CREATE TABLE education (id int,
                                     first_enroll text,
                                     first_enroll_term text,
                                     first_enroll_calendaryear int,
                                     high_completion_acadyr int,
                                     high_completion_label text,
                                     high_completion text,
                                     year7_enrolled int)''')

In [ ]:
#Load the education data to the database
education_df.to_sql('education',conn, if_exists = 'replace',index = False)

## 3. Join Datasets in SQL

In SQL, you have several options to join datasets. First, `LEFT JOIN` takes all the rows from the left dataset, and combines them with the matching rows from the right dataset. If there is no match in the right dataset, the result is `NaN` in the columns of the right dataset. A left join will ensure that all records from the left dataset are retained, even if there's no corresponding record in the right dataset. 

The example below illustrates how **left join** works by using *simplified versions* of the master data and the employment data. We can see that since person 3 is not in the employment data, after we left join the master data with the employment data, person 3 remains in the data but his year 7 earnings is `NaN`.

<text><center>**Master Data**</center></text>

|id|gender|birth_year|
|:--------:|:--------:|:--------|
|1|female|1996|
|2|male|1993|
|3|male|2000|

<text><center>**Employment Data**</center></text>

|id|year7_earnings|
|:--------:|:--------:|
|1|50000|
|2|60000|

<text><center>**Left Join the Master Data and the Employment Data**</center></text>

|id|gender|birth_year|year7_earnings|
|:--------:|:--------:|:--------:|:--------:|
|1|female|1996|50000|
|2|male|1993|60000|
|3|male|2000|NaN|

Let's do it in SQL. The `SELECT` statement is a fundamental SQL command used to fetch data from a database. It allows you to choose specific data from one or more tables by defining the columns you want to include. When you use `SELECT *`, the asterisk (*) is a wildcard character that represents 'all'. In the context of an SQL query, it means "select all columns". 

Can you spot the `LEFT JOIN` and `ON` in the next two lines? The SQL query is saying, "Select all columns from the *master* table, and then join it with the *employment* table, using `id` as the common column". 

The `pd.read_sql()` function is used to perform an SQL query, and to store the result in a new DataFrame named `master_employment_left_df`. The function takes two main arguments: the SQL query as a string, and the connection instance to the database.

In [ ]:
#Left join master data and employment data by using SQL query
master_employment_left_df = pd.read_sql('''SELECT * FROM master m 
                                           LEFT JOIN employment e 
                                           ON m.id = e.id''', conn)

Let's check the number of rows of `master_employment_left_df` and its columns. We can see that it has the same number of rows as `master_df`. However, many people have missing year 7 earnings.

In [ ]:
#Check number of rows
print('master_df has', master_df.shape[0], 'rows.')
print('master_employment_left_df has', master_employment_left_df.shape[0], 'rows.')

In [ ]:
#Check year 7 earnings missingness
master_employment_left_df['year7_earnings'].isna().sum()

Next, let's look at how **inner join** works. Inner join will only keep values that are matched in both datasets and drop all the rows that are not matched. The corresponding SQL command is `INNER JOIN`. Using the simplified example from earlier, the inner joined table would not have person 3's records.

<text><center>**Inner Join the Master Data and the Employment Data**</center></text>

|id|gender|birth_year|year7_earnings|
|:--------:|:--------:|:--------:|:--------:|
|1|female|1996|50000|
|2|male|1993|60000|

In [ ]:
#Inner join master data and employment data by using SQL query
master_employment_inner_df = pd.read_sql('''SELECT * FROM master m 
                                            INNER JOIN employment e 
                                            ON m.id = e.id''', conn)

We can see that by using `INNER JOIN`, the resulting DataFrame `master_employment_inner_df` has fewer rows than `master_df`. However, we do not have missing year 7 earnings in the `master_employment_inner_df`.

In [ ]:
#Check number of rows
print('master_df has', master_df.shape[0], 'rows.')
print('employment_df has', employment_df.shape[0], 'rows.')
print('master_employment_inner_df has', master_employment_inner_df.shape[0], 'rows.')

In [ ]:
#Check year 7 earnings missingness
master_employment_inner_df['year7_earnings'].isna().sum()

**When should you use left join and when should you use inner join?** It depends. If you want to study the whole population in the master data, you should always left join the master data with other datasets to ensure that you keep all the people in the master data. However, if you are only interested in people who have earnings in year 7, you can perform inner join or right join the master data with the employment data.

#### **Checkpoint 1: Join the Master Data and the Education Data**

Please join the master table with the education table by using SQL command `LEFT JOIN` and `INNER JOIN`. Check how the number of rows changes in the final DataFrames.

## 4. Data Linkage using Python

The `merge` function from the pandas library is a powerful tool in Python for joining or combining data from different datasets based on common columns. It replicates various types of SQL joins. SQL is often preferred for combining larger datasets or performing complex queries, mainly because of its efficiency. However, Python joins remain useful. Once data has been filtered or aggregated to a manageable size in SQL, Python's pandas library provides a more flexible and intuitive interface for further data manipulation and analysis.

In the code below, we use the `merge()` function to left join and inner join the master data and the employment data. In the `merge()` function, you need to define the DataFrame you want to join the `master_df` with, the join method in `how = `, and the common columns used for joining in `on = `.

In [ ]:
#Left Join in Python
master_emp_pd_left_df = master_df.merge(employment_df, how = 'left', on = 'id')

#Inner Join in Python
master_emp_pd_inner_df = master_df.merge(employment_df, how = 'inner', on = 'id')

Let us now compare the joining results by using `merge()` in Python with the results by using SQL commands. We can see that we get the exact same results.

In [ ]:
#Compare results
print('In SQL, after left joining master data with employment data, our final number of rows is', 
      master_employment_left_df.shape[0], '.')
print('In Python, after left joining master data with employment data, our final number of rows is', 
      master_emp_pd_left_df.shape[0], '.')

print('In SQL, after inner joining master data with employment data, our final number of rows is', 
      master_employment_inner_df.shape[0], '.')
print('In Python, after inner joining master data with employment data, our final number of rows is', 
      master_emp_pd_inner_df.shape[0], '.')

#### **Checkpoint 2: Right Join the Master Data and the Employment Data**

Please right join the master data, `master_df`, with the employment data, `employment_df`, by using pandas function `merge()`. Check how the number of rows changes in the final DataFrames.